# <center>Block 3c: Discrete choice estimation</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass on optimal transport and economic applications</center>
#### <center>With python code examples</center>
© 2018-2022 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/theteam).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass on optimal transport and economic applications, January 2022. https://github.com/math-econ-code/mec_optim

## References

* Savage, L. (1951). The theory of statistical decision. JASA.
* Bonnet, Fougère, Galichon, Poulhès (2021). Minimax estimation of hedonic models. Preprint.

## Loading the libraries

First, let's load the libraries we shall need.

In [1]:
import numpy as np
import os
import pandas as pd
import string as str
import math
import sys
import time
import scipy.sparse as spr
from scipy import optimize, special
# !python -m pip install -i https://pypi.gurobi.com gurobipy ## only if Gurobi not here
import gurobipy as grb

## Our data
We will go back to the dataset of Greene and Hensher (1997). As a reminder, 210 individuals are surveyed about their choice of travel mode between Sydney, Canberra and Melbourne, and the various costs (time and money) associated with each alternative. Therefore there are 840 = 4 x 210 observations, which we can stack into `travelmodedataset` a 3 dimensional array whose dimensions are mode,individual,dummy for choice+covariates.

Let's load the dataset and represent it conveniently in a similar fashion as in block 6:

In [2]:
#thepath = os.path.join(os.getcwd(),'data_mec_optim/demand_travelmode/')
thepath = 'https://raw.githubusercontent.com/math-econ-code/mec_optim_2021-01/master/data_mec_optim/demand_travelmode/'

travelmode =  pd.read_csv(thepath+'travelmodedata.csv')

travelmode['choice'] = np.where(travelmode['choice'] =='yes' , 1, 0)

nobs = travelmode.shape[0]
ncols = travelmode.shape[1]
nbchoices = 4
ninds = int(nobs/nbchoices)

muhat_i_y = travelmode['choice'].values.reshape(ninds,nbchoices).T
muhat_iy = muhat_i_y.flatten()

muhat_i_y = travelmode['choice'].values.reshape(ninds,4).T
muhat_iy = muhat_i_y.flatten()

s_y = travelmode.groupby(['mode']).mean()['choice'].to_frame().sort_index()

def two_d(X):
    return np.reshape(X,(X.size, 1))

# Estimation with no observable heterogeneity

Start with assuming that there is no observable heterogeneity, so the only observation we have at hand are the aggregate market shares $s_y$. Hence the systematic utility will be the same for every agent. However, we wish to write a parametric model for it, namely assume a knwon parametric form for the dependence of $U_y$ with respect to various observed characteristics associated with $y$.

Assume then that the utilities are parameterized as follows: $U = \Phi \beta$ where $\beta\in\mathbb{R}^{p}$ is a parameter, and $\Phi$ is a $\left\vert \mathcal{Y}\right\vert \times p$ matrix.

The log-likelihood function is given by

\begin{align*}
l\left(  \beta\right)  =N\sum_{y}\hat{s}_{y}\log\sigma_{y}\left(\Phi \beta\right)
\end{align*}

A common estimation method of $\beta$ is by maximum likelihood%

\begin{align*}
\max_{\beta}l\left(  \beta\right)  .
\end{align*}

MLE is statistically efficient; the problem is that the problem is not guaranteed to be convex, so there may be computational difficulties (e.g. local optima).

### MLE, logit case

In the logit case,

\begin{align*}
l\left(  \beta\right)  =N\left\{  \hat{s}^{\intercal}\Phi\beta-\log\sum_{y}\exp\left(  \Phi\beta\right)  _{y}\right\}
\end{align*}

so that the max-likehood amounts to

\begin{align*}
\max_{\beta}\left\{  \hat{s}^{\intercal} \Phi \beta-G\left( \Phi \beta\right)
_{y}\right\}
\end{align*}

whose value is the Legendre-Fenchel transform of $\beta\rightarrow G\left( \Phi \beta\right)$ evaluated at $\Phi ^{^{\intercal}}\hat{s}$.

Note that the vector $\Phi^{^{\intercal}}\hat{s}$ is the vector of empirical moments, which is a sufficient statistics in the logit model.

As a result, in the logit case, the MLE is a convex optimization problem, and it is therefore both statistically efficient and computationally efficient.

### Moment estimation

The previous remark will inspire an alternative procedure based on the moments statistics $\Phi^{^{\intercal}}\hat{s}$.

The social welfare is given in general by $W\left(  \beta\right) =G\left(  \Phi\beta\right)  $. One has $\partial_{\beta^{i}}W\left(\beta\right)  =\sum_{y}\sigma_{y}\left(  \Phi\beta\right)  \Phi_{yi}$, that is 

\begin{align*}
\nabla W\left(  \beta\right)  = \Phi^{\intercal}\sigma\left(  \Phi\beta\right)  ,
\end{align*}

which is the vector of predicted moments.

Therefore the program

\begin{align*}
\max_{\beta}\left\{  \hat{s}^{\intercal}\Phi\beta-G\left(  \Phi\beta\right)
_{y}\right\}
\end{align*}

picks up the parameter $\beta$ which matches the empirical moments $X^{^{\intercal}}\hat{s}$ with the predicted ones $\nabla W\left(\beta\right)  $. This procedure is not statistically efficient, but is computationally efficient becauses it arises from a convex optimization problem.

### Fixed temperature MLE

Back to the logit case. Recall we have

\begin{align*}
l\left(  \beta\right)  =N\left\{  \hat{s}^{\intercal}\Phi\beta-\log\sum_{y} \exp\left(  \Phi\beta\right)  _{y}\right\}
\end{align*}

Assume that we restrict ourselves to $\beta^{\top}z>0$. Then we can write $\beta=\theta/T$ where $T=1/\beta^{\top}z$ and $\theta=\beta T$. Call $\Theta=\left\{  \theta\in\mathbb{R}^{p},\theta^{\top}z=1\right\}  $, so that $\beta=\theta/T$ where $\theta\in\Theta$ and $T>0$. We have

\begin{align*}
l\left(  \theta,T\right)  =\frac{N}{T}\left\{  \hat{s}^{\intercal}
\Phi\theta-T\log\sum_{y}\exp\left(  \frac{\left(  \Phi\theta\right)  _{y}}{T}\right)  \right\}
\end{align*}

and we define the *fixed temperature maximum likelihood estimator* by

\begin{align*}
\theta\left(  T\right)  =\arg\max_{\theta}l\left(  \theta,T\right)
\end{align*}

 Note that $\theta\left(  T\right)  =\arg\max_{\theta\in\Theta}Tl\left(\theta,T\right)$ where

\begin{align*}
Tl\left(  \theta,T\right)  =N\left\{  \hat{s}^{\intercal}\Phi\theta-T\log\sum _{y}\exp\left(  \frac{\left(  \Phi\theta\right)  _{y}}{T}\right)  \right\}
\end{align*}

and we note that $Tl\left(  \theta,T\right)  \rightarrow N\left\{  \hat{s}^{\intercal}\Phi\theta-\max_{y\in\mathcal{Y}}\left\{  \left(  \Phi\theta\right)_{y}\right\}  \right\}  $ as $T\rightarrow0$.

We have

\begin{align*}
\frac{Tl\left(  \theta,T\right)  }{N}=\hat{s}^{\intercal}\Phi\theta-T\log\sum_{y}\exp\left(  \frac{\left(  \Phi\theta\right)  _{y}}{T}\right)
\end{align*}

Let $\theta\left(  0\right)  =\lim_{T\rightarrow0}\theta\left(T\right)  $. Calling $m\left(  \theta\right)  =\max_{y\in\mathcal{Y}}\left\{\left(  \Phi\theta\right)  _{y}\right\}  $, we have

\begin{align*}
\theta\left(  0\right)  \in\arg\max_{\theta}\left\{  \hat{s}^{\intercal}\Phi\theta-m\left(  \theta\right)  \right\},
\end{align*}

or

\begin{align*}
\theta\left(  0\right)  \in\arg\min_{\theta}\left\{  m\left(  \theta\right)-\hat{s}^{\intercal}\Phi\theta\right\},
\end{align*}

Calling $m\left(  \theta\right)  =\max_{y\in\mathcal{Y}}\left\{  \left(\Phi\theta\right)  _{y}\right\}  $, one has 

\begin{align*}
\theta\left(  T\right)  \in\arg\max\left\{  \hat{s}^{\intercal}\Phi\theta-m\left(  \theta\right)  -T\log\sum_{y}\exp\left(  \frac{\left(\Phi\theta\right)  _{y}-m\left(  \theta\right)  }{T}\right)  \right\}
\end{align*}


### Minimax-regret estimation

Note that

\begin{align*}
\theta\left(  0\right)  \in\arg\max\left\{  \hat{s}^{\intercal}\Phi\theta
-m\left(  \theta\right)  \right\}  .
\end{align*}

Define $R_{i}\left(  \theta,y\right)  =\left(  \Phi\theta\right)_{y}-\left(  \Phi\theta\right)  _{y_{i}}$ the regret associated with observation $i$ with respect to $y$. This is equal to the difference between the payoff given by $y$ and the payoff obtained under observation $i$, denoting $y_{i}$ the action taken in observation $i$. The max-regret associated with observation $i$ is therefore

\begin{align*}
\max_{y\in\mathcal{Y}}R_{i}\left(  \theta,y\right)  =\max_{y\in\mathcal{Y}}\left\{  \left(  \Phi\theta\right)_{y}-\left(  \Phi\theta\right)_{y_{i}}\right\}
\end{align*}

and the max-regret associated with the sample is $\frac{1}{N}\sum\max_{y\in\mathcal{Y}}\left\{  R_{i}\left(  \theta,y\right)  \right\}  $, that is $\max_{y\in\mathcal{Y}}\left\{  \left(  \Phi\theta\right)  _{y}\right\} - \hat{s}^{\intercal}X\theta$.

The minimax regret estimator

\begin{align*}
\hat{\theta}^{MMR}=\min_{\theta}\left\{  m\left(  \theta\right)  -\hat
{s}^{\intercal}\Phi\theta\right\}
\end{align*}

which has a linear programming fomulation

\begin{align*}
&  \min_{m,\theta}m-\hat{s}^{\intercal}\Phi\theta\\
s.t.~ &  m-\left(  \Phi\theta\right)  _{y}\geq\forall y\in\mathcal{Y}
\end{align*}

### Set-identification

Note that the set of $\theta$ that enter the solution to the problem above is not unique, but is a convex set. Denoting $V$ the value of program, we can look for bounds of $\theta^{\intercal}d$ for a chosen direction $d$ by

\begin{align*}
& \min_{\theta,m}/\max_{\theta,m}   \theta^{\intercal}d\\
s.t.~  &  m-\hat{s}^{\intercal}X\theta=V\\
&  m\geq\left(  \Phi\theta\right)_{y}, \quad \forall y\in\mathcal{Y}%
\end{align*}

## Link with exponential families and GLM

See class notes

# Estimation with observed heterogeneity

We now assume that we observe individual characteristics that are relevant for individual choices, that is $U_{iy}=\sum_k \Phi_{iyk} \beta_k$, or in matrix form
$$U = \Phi \beta,$$ where $\beta\in\mathbb{R}^{p}$ is a parameter, and $\Phi$ is a $\left(\left\vert \mathcal{I}\left\vert\right\vert\mathcal{Y}\right\vert \right) \times p$ matrix.

See class notes.

# Application

Back to the dataset:

In [3]:
Phi_iy_k = np.column_stack((np.kron(np.identity(4)[0:4,1:4],np.repeat(1, ninds).reshape(ninds,1)), - travelmode['travel'].values, - (travelmode['travel']*travelmode['income']).values, - travelmode['gcost'].values))

In [4]:
nbK = Phi_iy_k.shape[1]
phi_mean = Phi_iy_k.mean(axis = 0)
phi_stdev = Phi_iy_k.std(axis = 0, ddof = 1)
Phi_iy_k = ((Phi_iy_k - phi_mean).T/phi_stdev[:,None]).T

In [5]:
def log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    val = np.sum(np.multiply(Xtheta,muhat_iy))  - np.sum(max_i) - sigma * np.sum(np.log(d_i))

    return -val

In [6]:
def grad_log_likelihood(theta):
    nbK = np.asarray(theta).shape[0]
    Xtheta = Phi_iy_k.dot(theta)/sigma
    Xthetamat_iy = Xtheta.reshape(nbchoices, ninds).T
    max_i = np.amax(Xthetamat_iy, axis = 1)
    expPhi_iy = np.exp((Xthetamat_iy.T -max_i).T)
    d_i = np.sum(expPhi_iy, axis = 1)
    
    temp_mat = np.multiply(Phi_iy_k.T, expPhi_iy.T.flatten()).T
    list_temp = []
    for i in range(nbchoices):
        list_temp.append(temp_mat[i*ninds:(i+1)*ninds,])
    n_i_k = np.sum(list_temp,axis = 0)
    
    thegrad = muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten() - np.sum(n_i_k.T/d_i, axis = 1)

    return -thegrad

In [7]:
theta0 = np.repeat(0,nbK)
sigma = 1
outcome = optimize.minimize(log_likelihood,method = 'CG',jac = grad_log_likelihood, x0 = theta0)

In [8]:
outcome

     fun: 280.9103677250631
     jac: array([-1.59361683e-06, -4.35200363e-07, -1.85517164e-06,  1.90033509e-07,
        5.41492801e-07,  2.58135000e-07])
 message: 'Optimization terminated successfully.'
    nfev: 31
     nit: 16
    njev: 31
  status: 0
 success: True
       x: array([-0.02769993, -0.35847009,  0.01487751,  0.01416175,  0.11784738,
       -0.25344194])

In [9]:
temp_mle = 1 / outcome['x'][nbK - 1]
theta_mle = outcome['x']*temp_mle
print(temp_mle)
print(theta_mle)

-3.9456769452224876
[ 0.10929499  1.41440717 -0.05870185 -0.05587767 -0.46498768  1.        ]


In [10]:
lenobj = nbK+ninds
c = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1,ninds)))

m = grb.Model('lp')
x = m.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
m.setObjective(c @ x, grb.GRB.MAXIMIZE)
cstMat = spr.hstack((spr.csr_matrix(-Phi_iy_k), spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds))))
rhs = np.repeat(0,ninds*nbchoices)
m.addConstr(cstMat @ x >= rhs)
nbCstr = cstMat.shape[0]
const_2 = np.array([0]*(nbK - 1))
const_2 = np.append(const_2, 1)
const_2 = np.append(const_2 ,[0]*ninds)
m.addConstr(const_2 @ x == 1)
m.optimize()
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", m.objval)
    opt_x = m.getAttr('x')

Using license file C:\Users\Alfred\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 841 rows, 216 columns and 5881 nonzeros
Model fingerprint: 0xece1c49f
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [2e-01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 211 rows and 1 columns
Presolve time: 0.02s
Presolved: 630 rows, 215 columns, 2520 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     380   -1.3829488e+02   0.000000e+00   0.000000e+00      0s

Solved in 380 iterations and 0.05 seconds
Optimal objective -1.382948769e+02
Value of the problem (Gurobi) = -138.29487687593996


In [11]:
theta_lp = np.array(opt_x[:nbK])
print(theta_lp)
print(theta_mle)

[ 0.13949426  0.05032141  0.03395604 -0.72749334 -0.02506117  1.        ]
[ 0.10929499  1.41440717 -0.05870185 -0.05587767 -0.46498768  1.        ]


In [12]:
indMax=100
tempMax=temp_mle
outcomemat = np.zeros((indMax+1,nbK-1))

In [13]:
def log_likelihood_fixedtemp(subsetoftheta, *temp):
    val = log_likelihood(np.append(subsetoftheta, 1/temp[0]))
    
    return val

In [14]:
def grad_log_likelihood_fixedtemp(subsetoftheta, *temp):
    val = grad_log_likelihood(np.append(subsetoftheta, 1/temp[0]))[:-1]
    
    return val

In [15]:
outcomemat[0,:] = theta_lp[:-1]
iterMax = indMax+1
for k in range(2,iterMax+1,1):
    thetemp = tempMax * (k-1)/indMax
    outcomeFixedTemp = optimize.minimize(log_likelihood_fixedtemp,method = 'CG',jac = grad_log_likelihood_fixedtemp, args = (thetemp,),  x0 = theta0[:-1])
    outcomemat[k-1,:] = outcomeFixedTemp['x']*thetemp

In [16]:
outcomemat

array([[ 0.13949426,  0.05032141,  0.03395604, -0.72749334, -0.02506117],
       [ 0.14292345,  0.42333439, -0.06371649, -0.51242318, -0.1314772 ],
       [ 0.14482454,  0.42218714, -0.06321331, -0.50753462, -0.12936482],
       [ 0.14587841,  0.42033993, -0.06257653, -0.5042307 , -0.13058508],
       [ 0.14624026,  0.41848991, -0.06210426, -0.50100175, -0.13399352],
       [ 0.14642665,  0.41756089, -0.06139318, -0.49794246, -0.13827628],
       [ 0.14665808,  0.41780041, -0.06035416, -0.49509681, -0.14277605],
       [ 0.14695082,  0.41912092, -0.05907464, -0.4924095 , -0.14720936],
       [ 0.1472558 ,  0.42138082, -0.05767761, -0.48979281, -0.15145913],
       [ 0.1475228 ,  0.42447024, -0.05626612, -0.48716609, -0.1554836 ],
       [ 0.14771955,  0.42831363, -0.05491025, -0.48446871, -0.15928106],
       [ 0.147832  ,  0.43285517, -0.05365016, -0.48166091, -0.16287184],
       [ 0.14785899,  0.43804697, -0.05250383, -0.47872002, -0.16628727],
       [ 0.14780664,  0.44384285, -0.0

The zero-temperature estimator is:

In [17]:
print(outcomemat[1,:])

[ 0.14292345  0.42333439 -0.06371649 -0.51242318 -0.1314772 ]


The mle estimator is:

In [18]:
print(outcomemat[indMax,])

[ 0.10929494  1.41440716 -0.05870187 -0.0558777  -0.46498762]


In [19]:
nbB = 100
thetemp = 1
epsilon_biy = special.digamma(1) -np.log(-np.log(np.random.uniform(0,1,ninds*nbchoices*nbB)))
lenobj = ninds*nbB+nbK

newc = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1/nbB,ninds*nbB)))
newm = grb.Model('new_lp')
x = newm.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
newm.setObjective(newc @ x, grb.GRB.MAXIMIZE)
mat1 = spr.kron(-Phi_iy_k, two_d(np.repeat(1,nbB)))
mat2 = spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds*nbB))
newcstMat = spr.hstack((mat1, mat2))
rhs = epsilon_biy
newm.addConstr(newcstMat @ x >= rhs)
newm.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 84000 rows, 21006 columns and 588000 nonzeros
Model fingerprint: 0x3ae603a5
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 1e+01]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.45s
Presolved: 21006 rows, 84000 columns, 588000 nonzeros

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 1.260e+05
 Factor NZ  : 1.474e+05 (roughly 40 MBytes of memory)
 Factor Ops : 1.041e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.11855881e+02 -1.50058012e+00  1.08e+00 8.60e+00  5.15e+00     1s
   1  -1.28962387e+01 -3.90804245e+03  4.17e-03 1.78e-14  6.58e-02     1s
   2  -4.25062485e+01 -8.59730824e+02  7.15e-06 1.15e-14  9.7

In [20]:
if m.status == grb.GRB.Status.OPTIMAL:
    print("Value of the problem (Gurobi) =", newm.objval)
    opt_x = np.array(newm.getAttr('x'))
newtheta_lp = opt_x[:nbK] / opt_x[nbK-1]

Value of the problem (Gurobi) = -281.3157564831128


In [21]:
print(theta_mle)
print(newtheta_lp)

[ 0.10929499  1.41440717 -0.05870185 -0.05587767 -0.46498768  1.        ]
[ 0.11080115  1.51105502 -0.05999007 -0.07043508 -0.47497028  1.        ]


Finally probit

In [22]:
nbB = 100
thetemp = 1
epsilon_biy = np.random.normal(nbB*ninds*nbchoices)
lenobj = ninds*nbB+nbK

newc = np.concatenate((muhat_iy.reshape(1,nbchoices*ninds).dot(Phi_iy_k).flatten(),np.repeat(-1/nbB,ninds*nbB)))
newm = grb.Model('new_lp')
x = newm.addMVar(lenobj, name='x', lb=-grb.GRB.INFINITY)
newm.setObjective(newc @ x, grb.GRB.MAXIMIZE)
mat1 = spr.kron(-Phi_iy_k, two_d(np.repeat(1,nbB)))
mat2 = spr.kron(two_d(np.repeat(1,nbchoices)),spr.identity(ninds*nbB))
newcstMat = spr.hstack((mat1, mat2))
rhs = epsilon_biy
newm.addConstr(newcstMat @ x >= rhs)
newm.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 84000 rows, 21006 columns and 588000 nonzeros
Model fingerprint: 0x15ef26ea
Coefficient statistics:
  Matrix range     [3e-03, 5e+00]
  Objective range  [1e-02, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+04, 8e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.39s
Presolved: 21006 rows, 84000 columns, 588000 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.275e+03
 Factor NZ  : 1.869e+03
 Factor Ops : 2.260e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.64951762e+08 -1.76401410e+07  2.31e+00 1.05e-09  4.50e+00     1s
   1  -1.76401410e+07 -1.76404605e+07  1.20e-14 1.85e-10  3.80e-01     1s
   2  -1.76401410e+07 -1.76401414e+07  1.33e-15 2.34e-10  3.80e-04     1s
   3  -1.76401410